In [1]:
import pandas as pd
import json
from datetime import datetime
import numpy as np
from collections import Counter

In [3]:
with open('DataEngineeringQ2.json', 'r') as f:
    data = json.load(f)

df = pd.json_normalize(data)

print(f"Total number of records: {len(df)}")

Total number of records: 31


In [4]:
df['gender'] = df['patientDetails.gender']

gender_counts = df['gender'].value_counts()
print("\nOriginal gender distribution:")
print(gender_counts)

if not df['gender'].mode().empty and df['gender'].mode()[0] != "":
    mode_gender = df['gender'].mode()[0]
else:
    non_empty_genders = df['gender'][df['gender'] != ""]
    if len(non_empty_genders) > 0:
        mode_gender = non_empty_genders.mode()[0]
    else:
        mode_gender = "Unknown"  

print(f"Mode gender for imputation: '{mode_gender}'")

df['gender_imputed'] = df['gender'].replace('', mode_gender)
df['gender_imputed'] = df['gender_imputed'].fillna(mode_gender)

female_percentage = (df['gender_imputed'].str.lower() == 'female').mean() * 100
print(f"Percentage of female gender after imputation: {female_percentage:.2f}%")


Original gender distribution:
gender
M    11
F    10
      1
Name: count, dtype: int64
Mode gender for imputation: 'M'
Percentage of female gender after imputation: 0.00%


In [5]:
df['birthDate'] = df['patientDetails.birthDate']

# Function to calculate age from birthdate
def calculate_age(birth_date_str):
    if pd.isna(birth_date_str) or birth_date_str is None or birth_date_str == "":
        return np.nan
    
    try:
        # Parse the birth date (format depends on your data)
        # Assuming ISO format: 'YYYY-MM-DD'
        if isinstance(birth_date_str, str):
            birth_date = datetime.fromisoformat(birth_date_str.split('T')[0] if 'T' in birth_date_str else birth_date_str)
        else:
            # If it's already a datetime object
            birth_date = birth_date_str
        
        # Reference date: April 24, 2025
        reference_date = datetime(2025, 4, 24)
        
        # Calculate age
        age = reference_date.year - birth_date.year
        # Adjust age if birthday hasn't occurred yet this year
        if (reference_date.month, reference_date.day) < (birth_date.month, birth_date.day):
            age -= 1
            
        return age
    except Exception as e:
        print(f"Error calculating age for {birth_date_str}: {e}")
        return np.nan


In [6]:
df['age'] = df['birthDate'].apply(calculate_age)

# Define age groups
def assign_age_group(age):
    if pd.isna(age):
        return "Unknown"
    elif 0 <= age <= 12:
        return "Child"
    elif 13 <= age <= 19:
        return "Teen"
    elif 20 <= age <= 59:
        return "Adult"
    elif age >= 60:
        return "Senior"
    else:
        return "Unknown"

# Add ageGroup column
df['ageGroup'] = df['age'].apply(assign_age_group)

# Show distribution of age groups
age_group_counts = df['ageGroup'].value_counts()
print("\nAge group distribution:")
print(age_group_counts)


Age group distribution:
ageGroup
Adult      21
Unknown    10
Name: count, dtype: int64


In [7]:
medicines_list = []
for entry in data:
    # Get the list of medicines for this patient
    patient_id = entry['_id']
    if 'consultationData' in entry and 'medicines' in entry['consultationData']:
        for medicine in entry['consultationData']['medicines']:
            med_record = {
                'patient_id': patient_id,
                'medicineId': medicine.get('medicineId', ''),
                'medicineName': medicine.get('medicineName', ''),
                'isActive': medicine.get('isActive', False)
            }
            medicines_list.append(med_record)

medicines_df = pd.DataFrame(medicines_list)

In [8]:
if not medicines_df.empty:
    active_medicines = medicines_df[medicines_df['isActive'] == True]
    patient_medicine_counts = active_medicines.groupby('patient_id').size()
    
    # Calculate average medicines per patient
    if not patient_medicine_counts.empty:
        avg_medicines = patient_medicine_counts.mean()
        print(f"\nAverage number of active medicines prescribed per patient: {avg_medicines:.2f}")
    else:
        print("\nNo active medicines found in the dataset")
else:
    print("\nNo medicine data found in the dataset")


Average number of active medicines prescribed per patient: 2.09


In [9]:
if not medicines_df.empty:
    active_medicines = medicines_df[medicines_df['isActive'] == True]
    if not active_medicines.empty:
        medicine_counts = active_medicines['medicineName'].value_counts()
        
        print("\nTop prescribed medicines:")
        print(medicine_counts.head())
        
        # Get the 3rd most frequently prescribed medicine
        if len(medicine_counts) >= 3:
            third_most_common = medicine_counts.index[2]
            third_most_common_count = medicine_counts.iloc[2]
            print(f"\n3rd most frequently prescribed medicine: '{third_most_common}' (prescribed {third_most_common_count} times)")
        else:
            print("\nThere are fewer than 3 different medicines in the dataset")
    else:
        print("\nNo active medicines found in the dataset")


Top prescribed medicines:
medicineName
A    14
B    11
D     9
C     7
E     5
Name: count, dtype: int64

3rd most frequently prescribed medicine: 'D' (prescribed 9 times)


In [10]:
valid_ages = df['age'].dropna()
if not valid_ages.empty:
    average_age = valid_ages.mean()
    print(f"\nAverage age of patients as of April 24, 2025: {average_age:.2f} years")
    print(f"Based on {len(valid_ages)} patients with valid birth dates")
else:
    print("\nNo valid birth dates found in the dataset to calculate average age")


Average age of patients as of April 24, 2025: 34.81 years
Based on 21 patients with valid birth dates


In [11]:
df['phoneNumber'] = df['phoneNumber']

def is_valid_indian_phone_number(phone_number):
    """
    Validate if a phone number is a valid Indian phone number
    
    Rules:
    1. Can have +91 or 91 as prefix (optional)
    2. Must be between 6000000000-9999999999
    3. After removing prefix, should have 10 digits
    """
    if not phone_number or not isinstance(phone_number, str):
        return False
    
    # Remove spaces and any non-digit or '+' characters
    phone_number = ''.join(char for char in phone_number if char.isdigit() or char == '+')
    
    # Check for prefix and remove it
    if phone_number.startswith('+91'):
        phone_number = phone_number[3:]  # Remove '+91'
    elif phone_number.startswith('91'):
        phone_number = phone_number[2:]  # Remove '91'
    
    # Check if the number is 10 digits long
    if len(phone_number) != 10:
        return False
    
    # Check if the number starts with a digit between 6-9
    if not (phone_number[0] >= '6' and phone_number[0] <= '9'):
        return False
    
    # If all checks pass, return true
    return True

# Apply validation to phone numbers
df['isValidMobile'] = df['phoneNumber'].apply(is_valid_indian_phone_number)

# Count valid phone numbers
valid_phone_count = df['isValidMobile'].sum()
print(f"\nNumber of valid Indian phone numbers: {valid_phone_count} out of {len(df)}")


Number of valid Indian phone numbers: 18 out of 31


In [ ]:
import pandas as pd
import json
from datetime import datetime
import numpy as np
from collections import Counter
import re

# Load the JSON data
# Replace 'your_file.json' with your actual file path
with open('DataEngineeringQ2.json', 'r') as f:
    data = json.load(f)

# Convert to DataFrame for easier analysis
df = pd.json_normalize(data)

print(f"Total number of records: {len(df)}")

# ----------------- Question 1: Gender percentage after imputation -----------------
# Extract gender information
df['gender'] = df['patientDetails.gender']

# Handle gender imputation
gender_counts = df['gender'].value_counts()
print("\nOriginal gender distribution:")
print(gender_counts)

# Find the mode - if empty strings are the most common, use the next most common non-empty value
if not df['gender'].mode().empty and df['gender'].mode()[0] != "":
    mode_gender = df['gender'].mode()[0]
else:
    # Get non-empty values
    non_empty_genders = df['gender'][df['gender'] != ""]
    if len(non_empty_genders) > 0:
        mode_gender = non_empty_genders.mode()[0]
    else:
        mode_gender = "Unknown"  # Fallback if all values are empty

print(f"Mode gender for imputation: '{mode_gender}'")

# Fill missing or empty values with mode
df['gender_imputed'] = df['gender'].replace('', mode_gender)
df['gender_imputed'] = df['gender_imputed'].fillna(mode_gender)

# Calculate percentage of females after imputation
female_percentage = (df['gender_imputed'].str.lower() == 'female').mean() * 100
print(f"Percentage of female gender after imputation: {female_percentage:.2f}%")

# ----------------- Question 2: Age group categorization -----------------
# Extract birthDate for age calculation
df['birthDate'] = df['patientDetails.birthDate']

# Function to calculate age from birthdate
def calculate_age(birth_date_str):
    if pd.isna(birth_date_str) or birth_date_str is None or birth_date_str == "":
        return np.nan
    
    try:
        # Parse the birth date (format depends on your data)
        # Assuming ISO format: 'YYYY-MM-DD'
        if isinstance(birth_date_str, str):
            birth_date = datetime.fromisoformat(birth_date_str.split('T')[0] if 'T' in birth_date_str else birth_date_str)
        else:
            # If it's already a datetime object
            birth_date = birth_date_str
        
        # Reference date: April 24, 2025
        reference_date = datetime(2025, 4, 24)
        
        # Calculate age
        age = reference_date.year - birth_date.year
        # Adjust age if birthday hasn't occurred yet this year
        if (reference_date.month, reference_date.day) < (birth_date.month, birth_date.day):
            age -= 1
            
        return age
    except Exception as e:
        print(f"Error calculating age for {birth_date_str}: {e}")
        return np.nan

# Calculate age for each patient
df['age'] = df['birthDate'].apply(calculate_age)

# Define age groups
def assign_age_group(age):
    if pd.isna(age):
        return "Unknown"
    elif 0 <= age <= 12:
        return "Child"
    elif 13 <= age <= 19:
        return "Teen"
    elif 20 <= age <= 59:
        return "Adult"
    elif age >= 60:
        return "Senior"
    else:
        return "Unknown"

# Add ageGroup column
df['ageGroup'] = df['age'].apply(assign_age_group)

# Show distribution of age groups
age_group_counts = df['ageGroup'].value_counts()
print("\nAge group distribution:")
print(age_group_counts)

# ----------------- NEW QUESTION: Validate Indian phone numbers -----------------
# Extract phone number
df['phoneNumber'] = df['phoneNumber']

def is_valid_indian_phone_number(phone_number):
    """
    Validate if a phone number is a valid Indian phone number
    
    Rules:
    1. Can have +91 or 91 as prefix (optional)
    2. Must be between 6000000000-9999999999
    3. After removing prefix, should have 10 digits
    """
    if not phone_number or not isinstance(phone_number, str):
        return False
    
    # Remove spaces and any non-digit or '+' characters
    phone_number = ''.join(char for char in phone_number if char.isdigit() or char == '+')
    
    # Check for prefix and remove it
    if phone_number.startswith('+91'):
        phone_number = phone_number[3:]  # Remove '+91'
    elif phone_number.startswith('91'):
        phone_number = phone_number[2:]  # Remove '91'
    
    # Check if the number is 10 digits long
    if len(phone_number) != 10:
        return False
    
    # Check if the number starts with a digit between 6-9
    if not (phone_number[0] >= '6' and phone_number[0] <= '9'):
        return False
    
    # If all checks pass, return true
    return True

# Apply validation to phone numbers
df['isValidMobile'] = df['phoneNumber'].apply(is_valid_indian_phone_number)

# Count valid phone numbers
valid_phone_count = df['isValidMobile'].sum()
print(f"\nNumber of valid Indian phone numbers: {valid_phone_count} out of {len(df)}")

# ----------------- Question 3: Average medicines prescribed -----------------
# Extract medicine information
medicines_list = []
patient_medicine_count_dict = {}  # For correlation analysis

for entry in data:
    # Get the list of medicines for this patient
    patient_id = entry['_id']
    active_medicine_count = 0
    
    if 'consultationData' in entry and 'medicines' in entry['consultationData']:
        for medicine in entry['consultationData']['medicines']:
            is_active = medicine.get('isActive', False)
            med_record = {
                'patient_id': patient_id,
                'medicineId': medicine.get('medicineId', ''),
                'medicineName': medicine.get('medicineName', ''),
                'isActive': is_active
            }
            medicines_list.append(med_record)
            
            # Count active medicines for correlation analysis
            if is_active:
                active_medicine_count += 1
    
    # Store count of active medicines for this patient
    patient_medicine_count_dict[patient_id] = active_medicine_count

# Create medicines DataFrame
medicines_df = pd.DataFrame(medicines_list)

# Add medicine count to patient DataFrame
df['medicine_count'] = df['_id'].map(patient_medicine_count_dict)
df['medicine_count'] = df['medicine_count'].fillna(0).astype(int)

# Count active medicines per patient
if not medicines_df.empty:
    active_medicines = medicines_df[medicines_df['isActive'] == True]
    patient_medicine_counts = active_medicines.groupby('patient_id').size()
    
    # Calculate average medicines per patient
    if not patient_medicine_counts.empty:
        avg_medicines = patient_medicine_counts.mean()
        print(f"\nAverage number of active medicines prescribed per patient: {avg_medicines:.2f}")
    else:
        print("\nNo active medicines found in the dataset")
else:
    print("\nNo medicine data found in the dataset")

# ----------------- Question 4: 3rd most frequently prescribed medicine -----------------
# Count medicine frequency (only active prescriptions)
if not medicines_df.empty:
    active_medicines = medicines_df[medicines_df['isActive'] == True]
    if not active_medicines.empty:
        medicine_counts = active_medicines['medicineName'].value_counts()
        
        print("\nTop prescribed medicines:")
        print(medicine_counts.head())
        
        # Get the 3rd most frequently prescribed medicine
        if len(medicine_counts) >= 3:
            third_most_common = medicine_counts.index[2]
            third_most_common_count = medicine_counts.iloc[2]
            print(f"\n3rd most frequently prescribed medicine: '{third_most_common}' (prescribed {third_most_common_count} times)")
        else:
            print("\nThere are fewer than 3 different medicines in the dataset")
    else:
        print("\nNo active medicines found in the dataset")

# ----------------- Question 5: Average age of patients -----------------
# We've already calculated ages above using the reference date of April 24, 2025
valid_ages = df['age'].dropna()
if not valid_ages.empty:
    average_age = valid_ages.mean()
    print(f"\nAverage age of patients as of April 24, 2025: {average_age:.2f} years")
    print(f"Based on {len(valid_ages)} patients with valid birth dates")
else:
    print("\nNo valid birth dates found in the dataset to calculate average age")

# ----------------- NEW QUESTION: Correlation between medicine count and age -----------------
# Calculate Pearson correlation between age and medicine count
# First, filter to only include rows with valid age values
df_valid_age = df.dropna(subset=['age'])

if len(df_valid_age) > 0:
    correlation = df_valid_age['age'].corr(df_valid_age['medicine_count'], method='pearson')
    print(f"\nPearson correlation between patient age and number of prescribed medicines: {correlation:.4f}")
    
    # Additional interpretation for context
    if abs(correlation) < 0.3:
        strength = "weak"
    elif abs(correlation) < 0.7:
        strength = "moderate"
    else:
        strength = "strong"
        
    direction = "positive" if correlation > 0 else "negative"
    
    if abs(correlation) < 0.1:
        print("This indicates little to no linear relationship between age and medicine count.")
    else:
        print(f"This indicates a {strength} {direction} correlation: as age {'' if correlation > 0 else 'decreases'}, "
              f"the number of prescribed medicines tends to {'increase' if correlation > 0 else 'decrease'}.")
else:
    print("\nCannot calculate correlation: insufficient data with both valid age and medicine count.")

KeyError: 'gender'